In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_excel(r"D:\Pharma_Inventory_Optimization_Project\03_Model\Safety_Stock_Model.xlsx")

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df["Service_Level"] = 0.95

#i cannot calculate the ACV annual consumption value as i dont have the cost in the dataset

In [ ]:
from scipy.stats import norm

df["Z_value"] = norm.ppf(df["Service_Level"]).round(2)

#if i change service level percentage then this Z value will also change

In [ ]:
df.head()

In [ ]:
df.rename(columns={"StdDev_Demand": "Std_Demand"}, inplace=True)
df.rename(columns={"Average_Demand": "Avg_Demand"}, inplace=True)

In [ ]:
df["Avg_Demand"] = round(df["Avg_Demand"],2)
df["Std_Demand"] = round(df["Std_Demand"],2)

In [ ]:
df.head()

In [ ]:
df["Safety_Stock"] = round(df["Std_Demand"] * df["Z_value"] * np.sqrt(df["Lead_Time"]),0)
#square root of lead time
# I did round of till 0 cause the safety stock value can never be in points

In [ ]:
df["Reorder_Point"] = round((df["Avg_Demand"] * df["Lead_Time"]) + df["Safety_Stock"],0)

In [ ]:
df["CV_%"] = round((df["Std_Demand"] / df["Avg_Demand"]) * 100,0)

In [ ]:
df.tail()

In [ ]:
df.nlargest(1,"CV_%")

In [ ]:
df["CV_%"].count()

In [ ]:
df = df.sort_values(by="Avg_Demand", ascending=False).reset_index(drop=True)

In [ ]:
drug_names = [
"Paracetamol_500mg","Ibuprofen_400mg","Metformin_500mg","Amoxicillin_250mg",
"Azithromycin_500mg","Amlodipine_5mg","Pantoprazole_40mg","Aspirin_75mg",
"Telmisartan_40mg","Atorvastatin_10mg",
"Cefixime_200mg","Ciprofloxacin_500mg","Diclofenac_50mg","Glimepiride_2mg",
"Losartan_50mg","Rosuvastatin_20mg","Omeprazole_20mg","Levocetirizine_5mg",
"Montelukast_10mg","Sitagliptin_100mg",
"Doxycycline_100mg","Clopidogrel_75mg","Gabapentin_300mg","Tramadol_50mg",
"Sertraline_50mg","Escitalopram_10mg","Furosemide_40mg","Spironolactone_25mg",
"Metoprolol_50mg","Propranolol_40mg",
"Vitamin_D3_60000IU","Iron_Folic_Acid","Calcium_Carbonate","Ketoconazole_Cream",
"Mupirocin_Ointment","Loperamide_2mg","Ondansetron_4mg","Warfarin_5mg",
"Heparin_Injection","Cefuroxime_500mg","Amikacin_Injection",
"Prednisolone_10mg","Insulin_Glargine","Salbutamol_Inhaler",
"Budesonide_Inhaler","Hydroxychloroquine_200mg","Ranitidine_150mg","Enalapril_5mg",
"Clarithromycin_500mg","Rivaroxaban_10mg"
]
#The raw dataset had item IDs. I mapped them to product names using a reference list for better interpretability and dashboard presentation.

In [ ]:
df["Item_Name"] = drug_names[:len(df)]


In [ ]:
df = df.sort_values(by="Item").reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
df.isnull().value_counts()

In [ ]:
df.head()

In [ ]:
category_mapping = {

# Pain Relief
"Paracetamol_500mg": "Pain Relief",
"Ibuprofen_400mg": "Pain Relief",
"Diclofenac_50mg": "Pain Relief",
"Tramadol_50mg": "Pain Relief",

# Antibiotics
"Amoxicillin_250mg": "Antibiotic",
"Azithromycin_500mg": "Antibiotic",
"Cefixime_200mg": "Antibiotic",
"Ciprofloxacin_500mg": "Antibiotic",
"Doxycycline_100mg": "Antibiotic",
"Cefuroxime_500mg": "Antibiotic",
"Amikacin_Injection": "Antibiotic",
"Clarithromycin_500mg": "Antibiotic",

# Diabetes
"Metformin_500mg": "Diabetes",
"Glimepiride_2mg": "Diabetes",
"Sitagliptin_100mg": "Diabetes",
"Insulin_Glargine": "Diabetes",

# Cardiovascular
"Amlodipine_5mg": "Cardiovascular",
"Telmisartan_40mg": "Cardiovascular",
"Losartan_50mg": "Cardiovascular",
"Atorvastatin_10mg": "Cardiovascular",
"Rosuvastatin_20mg": "Cardiovascular",
"Metoprolol_50mg": "Cardiovascular",
"Propranolol_40mg": "Cardiovascular",
"Warfarin_5mg": "Cardiovascular",
"Heparin_Injection": "Cardiovascular",
"Clopidogrel_75mg": "Cardiovascular",
"Spironolactone_25mg": "Cardiovascular",
"Furosemide_40mg": "Cardiovascular",
"Enalapril_5mg": "Cardiovascular",
"Rivaroxaban_10mg": "Cardiovascular",
"Aspirin_75mg": "Cardiovascular",

# Gastro
"Pantoprazole_40mg": "Gastro",
"Omeprazole_20mg": "Gastro",
"Ranitidine_150mg": "Gastro",
"Loperamide_2mg": "Gastro",
"Ondansetron_4mg": "Gastro",

# Allergy
"Levocetirizine_5mg": "Allergy",
"Montelukast_10mg": "Allergy",

# Neurology / Psychiatric
"Gabapentin_300mg": "Neurology",
"Sertraline_50mg": "Psychiatric",
"Escitalopram_10mg": "Psychiatric",

# Supplements
"Vitamin_D3_60000IU": "Supplement",
"Iron_Folic_Acid": "Supplement",
"Calcium_Carbonate": "Supplement",

# Respiratory
"Salbutamol_Inhaler": "Respiratory",
"Budesonide_Inhaler": "Respiratory",

# Steroid / Immunology
"Prednisolone_10mg": "Steroid",
"Hydroxychloroquine_200mg": "Immunology",

# Dermatology
"Ketoconazole_Cream": "Dermatology",
"Mupirocin_Ointment": "Dermatology"
}

df["Category"] = df["Item_Name"].map(category_mapping)

In [ ]:
df = df[[
    "Item",
    "Item_Name",
    "Category",
    "Avg_Demand",
    "Std_Demand",
    "CV_%",
    "Lead_Time",
    "Service_Level",
    "Z_value",
    "Safety_Stock",
    "Reorder_Point"
]]
#reordering the columns

In [ ]:
df.head()

In [ ]:
df.to_csv("Safety_Stock.csv")